# Divvy Data (Feature) Engineering 
### Summary:
This module transforms travel history data into time sereies data of the station dock capacity
### Challenge & solutions
1. Data formats are not consistent -> Manually transform the data 
2. There are ',' in csv file -> Manually transform the data 
3. Data size is relatively big -> Use big data platforms to perform the aggregation 
4. Station_ID is inconsistent across the years -> Use Station_Name as Primary key instead 
### Methodology
1. Clean the data by aggregate into a single table 
2. Extract Day+Hour+10Min as Time_ID, group by Time_ID, Station_Name, aggregate by count
3. Create placeholding dataframe df_main 
4. Join each station into the placeholding dataframe to create a pivoted timeseries table, where each row is a time snapshot, each column is the time series of each station 
5. Merge In & Out to form Capacity Change
6. Finally, will compare the artificial delta agianst the actual station data and intepret the difference
### Data
- Source: https://www.divvybikes.com/system-data
- Description: This dataset includes individual Divvy bike sharing trips, including the origin, destination, timestamps, and rider type for each trip.

### Author: 
`Ryan Liao @2022/04/30 M.Sc Data Analytics Uchicago`

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import os
import numpy as np
from datetime import datetime
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import trange, tqdm
from datetime import datetime,timedelta


### 1. Clean the data by aggregate into a single table 


In [2]:
os.listdir('E:\\Data\\divvy\\TimeS')

['bike-sharing-best-bike-share-programs.jpg',
 'df_in_main_2019_2021.csv',
 'df_out_main_2019_2021.csv',
 'df_ts_balance_1719_ready.csv',
 'df_ts_balance_1721.csv',
 'df_ts_balance_1921.csv',
 'df_ts_balance_1921_ready.csv',
 'DivvyData_dock_hist1.csv',
 'Dock_ts.csv',
 'Dock_ts_flat.csv',
 'station_dock_ts_in1719_ts_pivot.csv',
 'station_dock_ts_out1719_ts_pivot.csv',
 'ts_IN.csv',
 'TS_in_1719_ts_pivot.csv',
 'ts_OUT.csv',
 'TS_OUT_1719_ts_pivot.csv']

In [3]:
os.listdir('E:\\Data\\divvy\\Travel_Hist')

['Divvy_Trips_2017_Q1.csv',
 'Divvy_Trips_2017_Q2.csv',
 'Divvy_Trips_2017_Q3.csv',
 'Divvy_Trips_2017_Q4.csv',
 'Divvy_Trips_2018_Q1.csv',
 'Divvy_Trips_2018_Q2.csv',
 'Divvy_Trips_2018_Q3.csv',
 'Divvy_Trips_2018_Q4.csv',
 'Divvy_Trips_2019_Q1.csv',
 'Divvy_Trips_2019_Q2.csv',
 'Divvy_Trips_2019_Q3.csv',
 'Divvy_Trips_2019_Q4.csv',
 'eve_attackers_ddl.sql',
 'eve_encounters_ddl.sql',
 'eve_items_ddl.sql',
 'eve_wars_ddl.sql',
 'README.txt',
 'README_2017_q1q2.txt',
 '__MACOSX']

In [8]:
# #Uniform the column names
# DATABASE = {}
# #unit_col = DATABASE['2017_Q1'].columns <- This depreciates
# for path in os.listdir('E:\\Data\\divvy\\Travel_Hist'):
#     file = "E:\\Data\\divvy\\Travel_Hist\\" + path 
#     if '.csv' in file:
#         name = path.split('.')[0][-7:]
#         df = pd.read_csv(file)
#         df = df.rename({i:j for i,j in zip(df.columns,unit_col)},axis=1)
#         df.start_time = df.start_time.apply(
#             lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S")
#         )
#         df.end_time = df.end_time.apply(
#                 lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S")
#             )
#         df.to_csv(file,index=False)
#         print(name)
#         #DATABASE[name] = pd.read_csv(file)

In [4]:
#Load into a single dataframe
DATABASE = {} 
for path in os.listdir('E:\\Data\\divvy\\Travel_Hist'):
    file = "E:\\Data\\divvy\\Travel_Hist\\" + path 
    if '.csv' in file:
        name = path.split('.')[0][-7:]
        df = pd.read_csv(file)
        DATABASE[name] = pd.read_csv(file)
        #print(name)
        #print(df.columns)

In [5]:
df_all = pd.DataFrame()
for db_name in DATABASE:
    df = DATABASE[db_name]
    df_all = pd.concat([df_all,df])

In [7]:
df_all.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,13518905,2017-03-31 23:59:07,2017-04-01 00:13:24,5292,857,66,Clinton St & Lake St,171,May St & Cullerton St,Subscriber,Male,1989.0
1,13518904,2017-03-31 23:56:25,2017-04-01 00:00:21,4408,236,199,Wabash Ave & Grand Ave,26,McClurg Ct & Illinois St,Subscriber,Male,1990.0
2,13518903,2017-03-31 23:55:33,2017-04-01 00:01:21,696,348,520,Greenview Ave & Jarvis Ave,432,Clark St & Lunt Ave,Subscriber,Female,1979.0
3,13518902,2017-03-31 23:54:46,2017-03-31 23:59:34,4915,288,110,Dearborn St & Erie St,142,McClurg Ct & Erie St,Subscriber,Male,1985.0
4,13518901,2017-03-31 23:53:33,2017-04-01 00:00:28,4247,415,327,Sheffield Ave & Webster Ave,331,Halsted St & Blackhawk St (*),Subscriber,Female,1989.0


In [1]:
#df_all.groupby(['to_station_id',	'to_station_name']).trip_id.count().to_csv('E:\Data\divvy/1719station_id_name.csv')

In [ ]:
#Examine column types to identify areas that requiring transformation
# start_time,end_time      -> Need to be mapped into datetime objects for easier manipulation
# tripduration             -> Supposed to be float but instead containing werid formatted string, such as "1,102"
df_all.dtypes

In [ ]:
df_all.start_time = df_all.start_time.apply(
    lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
)
df_all.end_time = df_all.end_time.apply(
        lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    )

In [ ]:
df_all.start_time.head().apply(lambda x:str(x)[:-4])

In [ ]:
df_all.tripduration = df_all.tripduration.apply(lambda x:float(str(x).replace(',','')))

In [ ]:
df_all.to_csv('E:\Data\divvy/Divvy_Trips_1719.csv')

### 2. Extract Day+Hour+10Min as Time_ID, group by Time_ID, Station_Name, aggregate by count


In [ ]:
DH10M_start = df_all.start_time.apply(lambda x:str(x)[:-4])
DH10M_end = df_all.end_time.apply(lambda x:str(x)[:-4])

In [ ]:
df_all['DH10M_start'] = DH10M_start
df_all['DH10M_end'] = DH10M_end

In [ ]:
# Already Done
ts_OUT =  pd.DataFrame(df_all.groupby(by=['DH10M_start','from_station_name']).trip_id.count())
ts_IN =   pd.DataFrame(df_all.groupby(by=['DH10M_end','to_station_name']).trip_id.count())
ts_IN = ts_IN.rename({'trip_id':'count_enter'},axis=1)
ts_OUT = ts_OUT.rename({'trip_id':'count_leave'},axis=1)
ts_OUT.DH10M_start = ts_OUT.DH10M_start.apply(lambda x: datetime.strptime(x+'0:00', "%Y-%m-%d %H:%M:%S"))
ts_OUT = ts_OUT.set_index('DH10M_start',drop=True)
ts_IN.DH10M_end = ts_IN.DH10M_end.apply(lambda x: datetime.strptime(x+'0:00', "%Y-%m-%d %H:%M:%S"))
ts_IN = ts_IN.set_index('DH10M_end',drop=True)
ts_IN.to_csv('E:\\Data\\divvy\\ts_IN.csv')
ts_OUT.to_csv('E:\\Data\\divvy\\ts_OUT.csv')


In [ ]:
ts_IN = pd.read_csv('E:\\Data\\divvy\\ts_IN.csv')
ts_OUT = pd.read_csv('E:\\Data\\divvy\\ts_OUT.csv')


### 3. Create placeholding dataframe df_main 


In [ ]:
def TS_gen(time_start,time_end,time_unit = timedelta(minutes=10)):
    """
    __summary__
    create a dataframe rangeing from time_start -> time_end
    """
    out_dict = {'time_stamp':[]}
    current_time = time_start
    while current_time < time_end:
        out_dict['time_stamp'].append(current_time)
        current_time += time_unit
    out = pd.DataFrame(out_dict)
    out.time_stamp = out.time_stamp.apply(lambda x: str(x))
    return out.set_index('time_stamp',drop=True)
    
def FillSpace(df,tablename,time_start,time_end,station_names,
              station_col_name,count_name,time_stamp_name,file_path):
    """
    __summary__
    Create a pivoted time series dataframe
    """
    ## Depreciated in newer version
    # df['time']=df[f'{tablename}.{prefix}_at_time'].apply(lambda x: f' {x}0:00')
    # df['start_timestamp'] = df[f'{tablename}.{prefix}_at_day'] + df['time']
    # df['start_timestamp'] = df['start_timestamp'].apply(
    #             lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    #         )
    print('data prepared')
    ########## Joining the tables ###########
    df_in_main = TS_gen(time_start,time_end)
    print('df_main generated,joining now...')
    count = 0 
    total = len(station_names)
    for station_name in tqdm(station_names):
        count+=1
        print(f'{station_name}|{count}/{total}')
        df_sub_set = df[df[f'{station_col_name}'] == station_name]
        df_sub_set = df_sub_set.set_index(time_stamp_name,drop=True)
        df_in_main = df_in_main.join(df_sub_set[count_name]).fillna(0).rename({count_name:station_name},axis=1)
    #Finally
    df_in_main.to_csv(f'{file_path}\\{tablename}_ts_pivot.csv')
    return df_in_main
##########
time_start = datetime(year =2017,month=1,day=1,hour=0,minute=20)
time_end = datetime(year =2019,month=9,day=30,hour=23,minute=50)
station_names = set(ts_IN.to_station_name.unique()).union(set(ts_OUT.from_station_name.unique()))

In [ ]:
df_ts_in = FillSpace(ts_IN,"TS_in_1719",time_start,time_end,station_names,
                     'to_station_name','count_enter','DH10M_end','E:\\Data\\divvy')

In [ ]:
df_ts_out = FillSpace(ts_OUT,"TS_OUT_1719",time_start,time_end,station_names,
          'from_station_name','count_leave','DH10M_start','E:\\Data\\divvy')

In [ ]:
df_ts_out = pd.read_csv('E:\\Data\\divvy\\TS_OUT_1719_ts_pivot.csv',index_col=0)

### 4. Join each station into the placeholding dataframe to create a pivoted timeseries table, where each row is a time snapshot, each column is the time series of each station 

### 5. Merge In & Out to form Capacity Change; Visualize time series

In [ ]:
df_ts_out['Damen Ave & Pierce Ave'].plot()
df_ts_in['Damen Ave & Pierce Ave'].plot()

In [ ]:
df_ts_balance = df_ts_in - df_ts_out

In [ ]:
df_ts_balance['Damen Ave & Pierce Ave'].plot(figsize=(16,9))

In [ ]:
df_ts_balance['Damen Ave & Pierce Ave'][:1000].plot(figsize=(16,9))

In [ ]:
df_ts_balance.to_csv('E:\\Data\\divvy\\df_ts_balance.csv')

In [ ]:
#This plots aggregated delta (assuming no staff manuvouring)
hour_row = 6 
day_row = 24 * hour_row
station_name = df_ts_in.columns[0:5]#'Lake Park Ave & 47th St'
df_ts_balance[station_name][:60*day_row].cumsum().plot(figsize=(16,9))
plt.title(f'{station_name} dock delta aggregated')
plt.xlabel('date')
plt.ylabel('dock delta')
# df_ts_in[station_name].cumsum().plot(figsize=(16,9))
# df_ts_out[station_name].cumsum().plot(figsize=(16,9))

In [ ]:
#This plots rolling cumsum, resetting each day. 
df_ts_balance = df_ts_balance.reset_index()
df_ts_balance.insert( 0, 'date' ,df_ts_balance.time_stamp.apply(lambda x:x.split(' ')[0]))
df_ts_balance = df_ts_balance.set_index('time_stamp',drop=True)
df_ts_balance_CS = df_ts_balance.groupby('date').cumsum()

In [ ]:
#Plotting
hour_row = 6 
day_row = 24 * hour_row
station_name = df_ts_in.columns[0:5]#'Lake Park Ave & 47th St'
df_ts_balance_CS[station_name][:30*day_row].plot(figsize=(16,9))
plt.title(f'{station_name}  dock delta daily reset')
plt.xlabel('date')
plt.ylabel('dock delta')

### Compare against actual data

In [ ]:
station_name = 'California Ave & Milwaukee Ave'

In [ ]:
df_ts_balance[station_name][:60*day_row].cumsum().plot(figsize=(16,9))
plt.title(f'{station_name} dock delta aggregated')
plt.xlabel('date')
plt.ylabel('dock delta')

In [ ]:
df_ts_balance_CS[station_name][:30*day_row].plot(figsize=(16,9))
plt.title(f'{station_name}  dock delta daily reset')
plt.xlabel('date')
plt.ylabel('dock delta')

In [ ]:
df_dockts_real = pd.read_csv('E:\\Data\\divvy\\TimeS\\DivvyData_dock_hist1.csv')
df_dockts_real.Timestamp = df_dockts_real.Timestamp.apply(
            lambda x: str(datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p"))
        )

In [ ]:
df_dockts_real = df_dockts_real.rename({'Total Docks':'Total_Docks'},axis=1)

In [ ]:
Num_days = 7
start = datetime.strptime('2017-01-01','%Y-%m-%d')
end = start + timedelta(days=Num_days)
df_dockts_real_subset = df_dockts_real.loc[(df_dockts_real['Timestamp']>=str(start)) & (df_dockts_real['Timestamp']<=str(end))].set_index('Timestamp')
df_dockts_real_subset['Available Docks'].plot()
base = df_dockts_real_subset['Available Docks'].iloc[0]
(base + -1*df_ts_balance['California Ave & Milwaukee Ave'][:Num_days*day_row].cumsum()).plot(figsize=(16,9))
plt.legend(['Acutal','agg_delta'])
plt.title(f'{station_name} dock delta aggregated')
plt.xlabel('date')
plt.ylabel('dock delta')
a = plt.gca()

In [ ]:
line_real = a.lines[0] # get the first line, there might be more
line_agg_delta = a.lines[1]
Real = line_real.get_ydata()
Artificial = line_agg_delta.get_ydata()[:len(Real)]
plt.figure(figsize=(16,9))
plt.plot( Real- Artificial)
plt.title('difference between real and artificial')

In [ ]:
Num_days = 14
start = datetime.strptime('2017-01-01','%Y-%m-%d')
end = start + timedelta(days=Num_days)
df_dockts_real_subset = df_dockts_real.loc[(df_dockts_real['Timestamp']>=str(start)) & (df_dockts_real['Timestamp']<=str(end))].set_index('Timestamp')
df_dockts_real_subset['Available Docks'].plot()
base = df_dockts_real_subset['Available Docks'].iloc[0]
(base + -1*df_ts_balance['California Ave & Milwaukee Ave'][:Num_days*day_row].cumsum()).plot(figsize=(16,9))
plt.legend(['Acutal','agg_delta'])
plt.title(f'{station_name} dock delta aggregated')
plt.xlabel('date')
plt.ylabel('dock delta')
a = plt.gca()

In [ ]:
line_real = a.lines[0] # get the first line, there might be more
line_agg_delta = a.lines[1]
Real = line_real.get_ydata()
Artificial = line_agg_delta.get_ydata()[:len(Real)]
plt.plot( Real- Artificial)

In [ ]:
Num_days = 30
start = datetime.strptime('2017-01-01','%Y-%m-%d')
end = start + timedelta(days=Num_days)
df_dockts_real_subset = df_dockts_real.loc[(df_dockts_real['Timestamp']>=str(start)) & (df_dockts_real['Timestamp']<=str(end))].set_index('Timestamp')
df_dockts_real_subset['Available Docks'].plot()
base = df_dockts_real_subset['Available Docks'].iloc[0]
(base + -1*df_ts_balance['California Ave & Milwaukee Ave'][:Num_days*day_row].cumsum()).plot(figsize=(16,9))
plt.legend(['Acutal','agg_delta'])
plt.title(f'{station_name} dock delta aggregated')
plt.xlabel('date')
plt.ylabel('dock delta')
###########
a = plt.gca()

In [ ]:
line_real = a.lines[0] # get the first line, there might be more
line_agg_delta = a.lines[1]
Real = line_real.get_ydata()
Artificial = line_agg_delta.get_ydata()[:len(Real)]
plt.plot( Real- Artificial)

#### Verifying this is no coincidence

In [ ]:
Num_days = 7
start = datetime.strptime('2021-05-25','%Y-%m-%d')
end = start + timedelta(days=Num_days)
df_dockts_real_subset = df_dockts_real.loc[(df_dockts_real['Timestamp']>=str(start)) & (df_dockts_real['Timestamp']<=str(end))].set_index('Timestamp')
df_dockts_real_subset['Available Docks'].plot()
base = df_dockts_real_subset['Available Docks'].iloc[0]
(base + -1*df_ts_balance['California Ave & Milwaukee Ave'][:Num_days*day_row].cumsum()).plot(figsize=(16,9))


plt.title(f'{station_name} dock delta aggregated')
plt.xlabel('date')
plt.ylabel('dock delta')
a = plt.gca()
#Cap
plt.plot([15 for i in range(len(a.lines[0].get_ydata()))])
plt.plot([-15 for i in range(len(a.lines[0].get_ydata()))])
plt.legend(['Acutal','agg_delta','upper-bound','lower-bound'])

In [ ]:
line_real = a.lines[0] # get the first line, there might be more
line_agg_delta = a.lines[1]
Real = line_real.get_ydata()
Artificial = line_agg_delta.get_ydata()[:len(Real)]
plt.plot( Real- Artificial)

### EDA Summary
**Observation**:  
- Taking Sample of Station (California Ave & Milwaukee Ave), I found that the Real TS and Artificial TS are very matching at first, and then diverges quickly. 
- There is also multiple small spkies along the way, but the difference plot demonstrates a step pattern   
**Explanation**:  
- Real data is capped by the total dock (15 in this station), whereas artificial data is not. 
- There are staff interventions, maneuver cars across stations 
- The artificial data only captures changes in 10 minute interval. Thus even ideally, it's only an estimate of the actual dock
- Noise, it's possible that there are some random events happens lead to the inaccuracy of dock or travel history data   

**Potential Metigation**:  
- When cap is reached (-15,+15), and there are still natural growth/decline, this means staff intervention has already happened 
- Ignore the staff intervention, since it's completly controlable from the company's point of view 
- Predict the staff itervention   

**Insights**:   
- Conduct small time interval (5-60 minutes) forecasting to help users navigates 
- Conduct daily forecasting to help Divvy scheduling staff intervention schedules to minimize Dock overflow / short of bikes
